In [7]:
import sys
sys.path.append('../') # Retrocede una carpeta atrás
from config.paths import SRC_DIR
import os
from dotenv import load_dotenv
load_dotenv()

from utils.scraping_utils import ScrapingUtils

In [8]:
urls = [
    {
    "url": "https://www.italika.mx/motocicleta-doble-proposito-italika-dm200-con-gps-azul-34006255-1300703478/p",
    "product_name": "Italika DM 200 GPS"
    }
]

In [9]:
import re

def remove_markdown_links(text):
    """
    Elimina enlaces markdown como:
    [texto](url)
    [![imagen](url)](url2)
    Deja solo el texto sin los links, elimina imágenes y títulos de links.
    """
    # Primero elimina los enlaces de imagen (con link exterior): [![alt](img)](link)
    pattern_img_link = re.compile(r'\[!\[.*?\]\(.*?\)\]\(.*?\)')
    text = pattern_img_link.sub('', text)
    # Después elimina los enlaces simples [texto](url)
    pattern_link = re.compile(r'\[([^\]]*?)\]\((?:[^)\s]+)(?:\s"[^"]*")?\)')
    text = pattern_link.sub(r'\1', text)
    # Finalmente elimina imágenes sueltas ![alt](url)
    pattern_img = re.compile(r'!\[.*?\]\(.*?\)')
    text = pattern_img.sub('', text)
    # Borra posibles líneas vacías generadas
    text = re.sub(r'\n+', '\n', text)
    return text.strip()


In [10]:
scraping_utils = ScrapingUtils(api_key=os.getenv("FIRECRAWL_API_KEY"))

In [11]:
for url in urls:
    data_scraped = scraping_utils.get_data_from_website(url["url"])
    markdown_no_urls = remove_markdown_links(data_scraped.markdown)
    with open(f"../data/scraped_data_downloaded/Italika/{url['product_name']}.html", "w", encoding="utf-8") as f:
        f.write(markdown_no_urls)

In [12]:
images_scraped = scraping_utils.get_images_from_website(url_to_extract_urls)

NameError: name 'url_to_extract_urls' is not defined

In [ ]:
images_scraped.json

{'imagenes': ['https://motoavanzada.mx/wp-content/uploads/2026/01/RAPTOR-70-GALERIA-PRINCIPAL-1024x1024.png',
  'https://motoavanzada.mx/wp-content/uploads/2026/01/1-2-scaled.png',
  'https://motoavanzada.mx/wp-content/uploads/2026/01/2-5-1024x1024.png',
  'https://motoavanzada.mx/wp-content/uploads/2026/01/4-3-1024x1024.png',
  'https://motoavanzada.mx/wp-content/uploads/2026/01/3-4-scaled.png',
  'https://motoavanzada.mx/wp-content/uploads/2026/01/5-4-1024x1024.png',
  'https://motoavanzada.mx/wp-content/uploads/2026/01/6-2-1024x1024.png']}

In [ ]:
import re
import json

def extract_urls_from_markdown(markdown_text_or_doc):
    """
    Extrae todas las URLs de un string en formato Markdown (enlaces y de imágenes).
    También puede recibir un objeto Document de Firecrawl.
    Ejemplo: [text](url), ![alt](url), [![alt](url)](url2)
    Lee correctamente represenentaciones tipo {'imagenes': [urls...]}
    """
    # Si recibimos directamente un dict tipo {'imagenes': [...]}
    if isinstance(markdown_text_or_doc, dict) and 'imagenes' in markdown_text_or_doc:
        return markdown_text_or_doc['imagenes']

    # Si es un objeto tipo Firecrawl Document
    if hasattr(markdown_text_or_doc, 'json'):
        # intentar leer .json como dict o str
        json_data = markdown_text_or_doc.json
        if isinstance(json_data, dict) and 'imagenes' in json_data:
            return json_data['imagenes']
        if isinstance(json_data, str):
            try:
                as_dict = json.loads(json_data)
                if isinstance(as_dict, dict) and 'imagenes' in as_dict:
                    return as_dict['imagenes']
            except Exception:
                pass

    # Si tiene markdown, úsalo
    if hasattr(markdown_text_or_doc, 'markdown'):
        markdown_text = markdown_text_or_doc.markdown or ""
    elif isinstance(markdown_text_or_doc, str):
        markdown_text = markdown_text_or_doc
    else:
        markdown_text = str(markdown_text_or_doc) if markdown_text_or_doc else ""

    # Si no hay markdown, retornar lista vacía
    if not markdown_text:
        return []

    # Busca urls dentro de paréntesis después de ] o ) o ![
    # Esto cubrirá casos como: [text](url), ![alt](url), [![alt](url)](url2)
    url_pattern = r'\((https?://[^\s)]+|/[^)\s]+)\)'
    urls = re.findall(url_pattern, markdown_text)
    return urls

In [ ]:
urls = extract_urls_from_markdown(images_scraped)
urls

['https://motoavanzada.mx/wp-content/uploads/2026/01/RAPTOR-70-GALERIA-PRINCIPAL-1024x1024.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/1-2-scaled.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/2-5-1024x1024.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/4-3-1024x1024.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/3-4-scaled.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/5-4-1024x1024.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/6-2-1024x1024.png']

## CAMBIAR LO QUE SE BUSCA DE LOS MULTIMEDIA

In [ ]:
# #
# # ! Asegurate de cambiar a lo que quieres buscar
# url_list = []
# text_list_to_search = ["/wp-content/uploads/vento"]
# for url in urls:
#     if any(text in url for text in text_list_to_search):
#         url_list.append(url)
# url_list

In [ ]:
data_scraped = scraping_utils.get_data_from_website(url_to_extract_urls)

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(data_scraped.html, "html.parser")
title_element = soup.find(class_="product_title entry-title elementor-heading-title elementor-size-default")
product_title = title_element.get_text(strip=True) if title_element else None
product_title

In [ ]:
import re

def extract_image_and_pdf_links(markdown_text):
    """
    Extrae todos los links de imágenes y PDFs que aparecen en el markdown.
    """
    # Extensiones de imagen y PDF a buscar
    image_extensions = ['.webp', '.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.svg']
    pdf_extensions = ['.pdf']
    all_extensions = image_extensions + pdf_extensions

    # Busca los links en formato ![](url) y [nombre](url)
    pattern = r'\((https?://[^\s)]+|/[^)\s]+)\)'
    matches = re.findall(pattern, markdown_text)

    # Filtra links que terminen con alguna de las extensiones buscadas
    links = [link for link in matches if any(link.lower().endswith(ext) for ext in all_extensions)]
    return links

links = extract_image_and_pdf_links(data_scraped.markdown)

In [ ]:
imagenes = []
pdfs = []
for item in links:
    image_extensions = [".webp", ".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".svg"]
    if any(ext in item.lower() for ext in image_extensions):
        if '300x300' not in item:
            imagenes.append(item)
    elif ".pdf" in item:
        pdfs.append(item)

# Eliminar duplicados
imagenes = list(set(imagenes))
pdfs = list(set(pdfs))

In [ ]:
pdfs

[]

In [ ]:
urls

['https://motoavanzada.mx/wp-content/uploads/2026/01/RAPTOR-70-GALERIA-PRINCIPAL-1024x1024.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/1-2-scaled.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/2-5-1024x1024.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/4-3-1024x1024.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/3-4-scaled.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/5-4-1024x1024.png',
 'https://motoavanzada.mx/wp-content/uploads/2026/01/6-2-1024x1024.png']

In [ ]:
import os
import requests

def descargar_archivos(lista_urls, carpeta_destino):
    """
    Descarga archivos desde una lista de URLs y los guarda en la carpeta de destino.
    Si se recibe un error 406, lo intenta de nuevo enviando headers de navegador.
    """
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Headers tipo navegador para evadir errores como 406
    browser_headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Referer': 'https://fratelliglobal.com/',
    }

    for url in lista_urls:
        try:
            nombre_archivo = url.split("/")[-1]
            ruta_completa = os.path.join(carpeta_destino, nombre_archivo)

            # Si ya existe, no descargar de nuevo
            if os.path.exists(ruta_completa):
                print(f"Archivo ya existe: {nombre_archivo}")
                continue

            try:
                response = requests.get(url, stream=True, timeout=20)
                response.raise_for_status()
            except requests.exceptions.HTTPError as e:
                if response.status_code == 406:
                    print(f"406 recibido para {url}, reintentando con headers de navegador...")
                    response = requests.get(url, stream=True, timeout=20, headers=browser_headers)
                    response.raise_for_status()
                else:
                    raise

            with open(ruta_completa, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:  # filtra paquetes vacíos
                        f.write(chunk)
            print(f"Descargado: {nombre_archivo}")
        except Exception as e:
            print(f"Error al descargar {url}: {e}")


In [ ]:
carpeta_destino = f"{SRC_DIR}/data/scraped_data_downloaded/raptor_70"
# descargar_archivos(pdfs, carpeta_destino)
descargar_archivos(urls, carpeta_destino)

Descargado: RAPTOR-70-GALERIA-PRINCIPAL-1024x1024.png
Descargado: 1-2-scaled.png
Descargado: 2-5-1024x1024.png
Descargado: 4-3-1024x1024.png
Descargado: 3-4-scaled.png
Descargado: 5-4-1024x1024.png
Descargado: 6-2-1024x1024.png
